In [11]:

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs =100

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [12]:

all_transforms = transforms.Compose([transforms.Resize((32,32)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                          std=[0.2023, 0.1994, 0.2010])
                                     ])
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = True,
                                             transform = all_transforms,
                                             download = True)

test_dataset = torchvision.datasets.CIFAR10(root = './data',
                                            train = False,
                                            transform = all_transforms,
                                            download=True)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
class ConvNeuralNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
      
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [14]:
model = ConvNeuralNet(num_classes).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  


In [15]:

for epoch in range(num_epochs):
  correct = 0
  total = 0
  for i, (images, labels) in enumerate(train_loader):  
        
        images = images.to(device)
        labels = labels.to(device)
        
        output = model(images)
        loss = criterion(output, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _,predicted = torch.max(output.data,1)
        total +=labels.size(0)
        correct += (predicted == labels).sum().item()
        

  print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {}% '.format(epoch+1, num_epochs, loss.item(),100 * correct / total))

         


Epoch [1/100], Loss: 2.2889, Accuracy: 16.024% 
Epoch [2/100], Loss: 2.0156, Accuracy: 22.912% 
Epoch [3/100], Loss: 1.7754, Accuracy: 27.958% 
Epoch [4/100], Loss: 1.7572, Accuracy: 31.572% 
Epoch [5/100], Loss: 1.8192, Accuracy: 34.43% 
Epoch [6/100], Loss: 1.7226, Accuracy: 37.184% 
Epoch [7/100], Loss: 1.8355, Accuracy: 39.432% 
Epoch [8/100], Loss: 1.2922, Accuracy: 41.374% 
Epoch [9/100], Loss: 1.5973, Accuracy: 43.224% 
Epoch [10/100], Loss: 1.6398, Accuracy: 44.776% 
Epoch [11/100], Loss: 1.7765, Accuracy: 46.076% 
Epoch [12/100], Loss: 1.3770, Accuracy: 47.244% 
Epoch [13/100], Loss: 1.3844, Accuracy: 48.21% 
Epoch [14/100], Loss: 1.8324, Accuracy: 49.33% 
Epoch [15/100], Loss: 1.5889, Accuracy: 50.224% 
Epoch [16/100], Loss: 1.2546, Accuracy: 50.816% 
Epoch [17/100], Loss: 1.7440, Accuracy: 51.714% 
Epoch [18/100], Loss: 1.5028, Accuracy: 52.416% 
Epoch [19/100], Loss: 1.6066, Accuracy: 53.204% 
Epoch [20/100], Loss: 1.5408, Accuracy: 53.838% 
Epoch [21/100], Loss: 1.4751, Ac

In [16]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network : {} %'.format(100 * correct / total))



Accuracy of the network : 64.17 %
